<a href="https://colab.research.google.com/github/SilverSurferClash/Binary_classification/blob/main/march21_base_model_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


In [5]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 644 kB/s 


In [6]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


In [3]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 4.8 MB/s 
     |████████████████████████████████| 210 kB 28.7 MB/s 
     |████████████████████████████████| 81 kB 7.4 MB/s 
     |████████████████████████████████| 78 kB 5.8 MB/s 
     |████████████████████████████████| 50 kB 4.7 MB/s 
     |████████████████████████████████| 147 kB 26.7 MB/s 
     |████████████████████████████████| 112 kB 41.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=62b5ff839158fa075938330ebbf4e1ad2d15841955ff77409558030a7994bda6
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0


In [7]:
import os
import time
from pathlib import Path
import sys
import pickle
import joblib

#Import data manipulation libaries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#import ploting libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Splitting the data and cross-validation
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score,KFold, GroupKFold, StratifiedKFold

#Metrics
#from sklearn.metrics import classification_report,accuracy_score , roc_auc_score
from sklearn.metrics import mean_squared_error, roc_auc_score
#Classifiers
import xgboost as xgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

#Feature engineerring
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_transformer

#Encoding

import category_encoders as ce

In [8]:
os.getcwd()

'/content'

In [9]:
os.chdir("/content/drive/MyDrive/Colab_Notebooks/Feb_21")

In [10]:
#Optuna optimization

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [11]:
train_import = pd.read_csv("train.csv")
test_import = pd.read_csv("test.csv")

In [12]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
                # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True
                # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)
                          # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [13]:
#reduce_mem_usage(train_import)

In [14]:
# List of colums based on their function
# Target
target_label  = "target"
# List of all features
all_feature = [col for col in train_import.columns if col not in ["target", "id"]]
# List of numeric features
all_numeric = [col for col in all_feature if train_import[col].dtype == "float64"]
# List of all categorical features
all_categorical = [col for col in all_feature if train_import[col].dtype == "object"]

high_cardinal = "cat10"

low_cardinal = ['cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9',
 'cat11',
 'cat12',
 'cat13',
 'cat14',
 'cat15',
 'cat16',
 'cat17',
 'cat18']

In [15]:
#Create the training feature matrix, the training feature labels and the same for the
#validation set
train = train_import.sample(frac = 0.1).copy().reset_index()
train_X = train.loc[:, all_feature]
train_y = train.loc[:, target_label].values
test_X = test_import.loc[:, all_feature]
train_X.shape, train_y.shape, test_X.shape

((30000, 30), (30000,), (200000, 30))

In [16]:
# Helper Code to make the code adaptible for the future
train_split_X = train_X.copy()
train_split_y = pd.DataFrame(train_y.copy())

In [17]:
#Set up validation scheme
n_folds = 2
splitter = StratifiedKFold(n_splits=n_folds)

In [18]:
preprocess_continuous = Pipeline(steps=[('scaler', StandardScaler())])

preprocess_low_cardinal = Pipeline(steps=[('low_card',
ce.OneHotEncoder(cols=low_cardinal))])

preprocess_high_cardinality = Pipeline(steps=[('high_cardinal',
ce.TargetEncoder(cols = high_cardinal))])

/usr/local/lib/python3.8/dist-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/usr/local/lib/python3.8/dist-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [19]:
ct = ColumnTransformer(
transformers=[('continuous', #A
preprocess_continuous, all_numeric),
('low_card', #B
preprocess_low_cardinal,low_cardinal),
   ("high_card", 
   preprocess_high_cardinality, high_cardinal)],
remainder='passthrough')

In [20]:
model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, tree_method = 'gpu_hist' )

In [21]:
#model = LGBMClassifier(device = 'gpu')

In [22]:
#model = LogisticRegression(max_iter=100)

In [23]:
#model = CatBoostClassifier(loss_function="Logloss",eval_metric="AUC",task_type="GPU")

In [26]:
#Optuna XGB
def objective(trial,data=train_split_X,target=train_split_y):
    
    train_x, val_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)


    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500, 50),
        'max_depth': trial.suggest_int('max_depth', 2,15,1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 100,5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 100.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 100.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 100.0),
        
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log=True),
    }
    Xtrain_prepocessed = ct.fit_transform(train_x, train_y.values.ravel())
    Xval_preprocessed = ct.transform(val_x)
    model = xgb.XGBClassifier(**param)
    model.fit(Xtrain_prepocessed, train_y.values.ravel(),
             verbose = False,
             eval_set = [(Xtrain_prepocessed, train_y.values.ravel()), (Xval_preprocessed, test_y.values.ravel())],
             eval_metric = "auc",
             early_stopping_rounds = 100)

    

    valid_pred = model.predict_proba(Xval_preprocessed)[:,1]
    valid_score = roc_auc_score(test_y, valid_pred)

    return valid_score
    


In [27]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

[I 2022-12-25 19:16:27,122] A new study created in memory with name: no-name-91f2ffbd-3e45-427a-becd-041201296778
<ipython-input-26-7593a2a3afbc>:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma', 1e-8, 100.0),
<ipython-input-26-7593a2a3afbc>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-8, 100.0),
<ipython-input-26-7593a2a3afbc>:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lam

In [29]:
best_xgb_params = study.best_params
best_xgb_params 

{'n_estimators': 800,
 'max_depth': 12,
 'min_child_weight': 11,
 'subsample': 0.9213389538994615,
 'colsample_bytree': 0.542818257565404,
 'gamma': 5.294309545680173e-06,
 'alpha': 3.662928103751163e-07,
 'lambda': 0.00040532307798668614,
 'learning_rate': 0.00561752688209016}

In [30]:

train_all_preprocessed = ct.fit_transform(train_split_X, train_split_y)
test_all_preprocessed = ct.transform(test_X)

/usr/local/lib/python3.8/dist-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/usr/local/lib/python3.8/dist-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [31]:
model = xgb.XGBClassifier(**best_xgb_params)

In [34]:
model.fit(train_all_preprocessed, train_split_y.values.ravel())

XGBClassifier(alpha=3.662928103751163e-07, colsample_bytree=0.542818257565404,
              gamma=5.294309545680173e-06, lambda=0.00040532307798668614,
              learning_rate=0.00561752688209016, max_depth=12,
              min_child_weight=11, n_estimators=800,
              subsample=0.9213389538994615)

In [35]:
xgb_pickle_name = "best_xgb_params_feb-2021.pkl"
with open(xgb_pickle_name, "wb") as file:
    pickle.dump(best_xgb_params, file)

In [ ]:
#Ideas: separte lists for binary, ordinal and nominal categoroes 